In [9]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from dotenv import load_dotenv

# load the environment variables
load_dotenv()

# get API key from environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# create a retrieval QA object
chat_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# create an openai embeddings object
embedding_provider = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# create a vector store object
movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://44.201.38.129:7687",
    username="neo4j",
    password="journals-cupful-lot",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

# create a graph object, including connection to the neo4j database
graph = Neo4jGraph(
    url="bolt://44.201.38.129:7687",
    username="neo4j",
    password="journals-cupful-lot"
)

plot_retriever = RetrievalQA.from_llm(
    llm=chat_llm,
    retriever=movie_plot_vector.as_retriever(),
    verbose=True,
    return_source_documents=True
)

In [11]:
import pprint
result = plot_retriever.invoke(
    {"query": "A movie where a mission to the moon goes wrong, starrin Tom Hanks"}
)

pprint.pprint(result)




> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'A movie where a mission to the moon goes wrong, starrin Tom Hanks',
 'result': 'The movie you are referring to is "Apollo 13" starring Tom Hanks. '
           'It depicts the real-life mission where NASA had to bring the '
           'damaged spacecraft and its crew safely back to Earth.',
 'source_documents': [Document(page_content='Humanity finds a mysterious, obviously artificial object buried beneath the Lunar surface and, with the intelligent computer H.A.L. 9000, sets off on a quest.', metadata={'budget': 12000000, 'movieId': '924', 'tmdbId': '62', 'imdbVotes': 407650, 'runtime': 149, 'countries': ['USA', ' UK'], 'imdbId': '0062622', 'url': 'https://themoviedb.org/movie/62', 'released': '1968-05-15', 'languages': ['English', ' Russian'], 'imdbRating': 8.3, 'title': '2001: A Space Odyssey', 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/zmmYdPa8Lxx999Af9vnVP4XQ1V6.jpg', 'year': 1968, 'revenue': 68700000}

In [3]:
print(graph.schema)

Node properties are the following:
Movie {url: STRING, runtime: INTEGER, revenue: INTEGER, budget: INTEGER, embedding: LIST, imdbRating: FLOAT, released: STRING, countries: LIST, languages: LIST, plot: STRING, imdbVotes: INTEGER, imdbId: STRING, year: INTEGER, poster: STRING, movieId: STRING, tmdbId: STRING, title: STRING},Genre {name: STRING},User {userId: STRING, name: STRING},Actor {url: STRING, died: DATE, born: DATE, imdbId: STRING, name: STRING, tmdbId: STRING, bornIn: STRING, bio: STRING, poster: STRING},Director {url: STRING, bornIn: STRING, bio: STRING, died: DATE, born: DATE, imdbId: STRING, name: STRING, poster: STRING, tmdbId: STRING},Person {url: STRING, bornIn: STRING, bio: STRING, died: DATE, born: DATE, imdbId: STRING, name: STRING, poster: STRING, tmdbId: STRING}
Relationship properties are the following:
RATED {rating: FLOAT, timestamp: INTEGER},ACTED_IN {role: STRING},DIRECTED {role: STRING}
The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:User)-

In [4]:
result = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.")
for doc in result:
    print(doc.metadata["title"], "-", doc.page_content)

Coneheads - Aliens with conical crania crash land on Earth.
Aliens - The planet from Alien (1979) has been colonized, but contact is lost. This time, the rescue team has impressive firepower, but will it be enough?
Independence Day (a.k.a. ID4) - The aliens are coming and their goal is to invade and destroy Earth. Fighting superior technology, mankind's best weapon is the will to survive.
Arrival, The - Zane, an astronomer, discovers intelligent alien life. But the aliens are keeping a deadly secret, and will do anything to stop Zane from learning it.


In [5]:
result = movie_plot_vector.similarity_search("A movie about a girl and a horse.")
for doc in result:
    print(doc.metadata["title"], "-", doc.page_content)

Black Beauty - The fates of horses, and the people who own and command them, are revealed as Black Beauty narrates the circle of his life.
Little Princess, A - A young girl is relegated to servitude at a boarding school when her father goes missing and is presumed dead.
Horseman on the Roof, The (Hussard sur le toit, Le) - In a time of war and disease, a young officer gallantly tries to help a young woman find her husband.
So Dear to My Heart - This heartwarming classic tells the tale of a country boy who adopts a mischevious black lamb and learns valuable lessons about love and dedication.
